In [240]:

%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint
from joblib import dump, load

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd
from multiprocessing import Pool

import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from rapidfuzz import process, fuzz
from sklearn.neural_network import MLPClassifier

from difflib import SequenceMatcher
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'

r = redis.Redis(host="127.0.0.1", port=6379, db=0)


from sqlalchemy import event,create_engine,types
driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
# from register_check_prod import RegisterCheck
# from pprint import pprint
# m = RegisterCheck()


# import time
# start_time = time.time()
# pprint(m.score_user('ewauuacuwcwac@gmail.com', 'eeee', '10.1.77.18', 'adawdawddaw'))
# print("--- %s seconds ---" % (time.time() - start_time))

In [242]:
def encode(s):
    
    i = 0
    t = ''
    while i < len(s):
        if s[i].isupper():
            t += 'U'
        elif s[i].islower():
            t += 'L'
        elif s[i].isdigit():
            t += 'D'
        else:
            t += 'O'
        i += 1
        
    return t

def short_encode(s):
    i = 0
    s = encode(s)
    curr = ''
    t = ''
    
    while i < len(s):
        if curr != s[i]:
            t += s[i]
            curr = s[i]
        
        i += 1
    return t

def score_sus_email(s):

    email_input = s
    email_input = email_input.split('@')[0]
    
    
    
    pat_count_digit = encode(email_input).count('D')
    pat_encode = short_encode(email_input)
    
    if pat_encode == 'LD' and pat_count_digit >= 4:
        return 1
    elif pat_encode == 'D':
        return 1
    return 0
    
score_sus_email('123312313@gmail.com')

1

In [243]:
# from gibberish_detector import detector

# path_detector_indo = '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/indo_news.model'
# Detector = detector.create_from_model(
#     path_detector_indo
# )

# def score_gibberish(input_email):
#     split_email = input_email.split('@')[0]
    
#     flag_gibberish1 = False
    
#     try:
#         if Detector.is_gibberish(split_email):
#             flag_gibberish1 = True
#     except Exception as e:
#         pass


#     if flag_gibberish1:
#         return 1
#     else:
#         return 0

# email = 'qywyeqeyqw123@gmail.com'
# score_gibberish(email)

In [244]:
from gibberish_detector import detector

path_detector_indo = '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/indo_news.model.test'
Detector = detector.create_from_model(
    path_detector_indo
)

def score_gibberish(input_email):
    split_email = input_email.split('@')[0]
    
    score_gib = Detector.calculate_probability_of_being_gibberish(split_email)
    
    return score_gib

email = 'fadmawdwa@gmail.com'
score_gibberish(email)



3.7002451204535642

In [31]:
alfagift_member = pd.read_json('/home/server/mongo_export/alfagift_member_01jan22.json',
                              dtype='object')
alfagift_member['sapa'] = alfagift_member['sapa'].fillna('-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: x.get('store_code') \
                                                        if x != '-' else '-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: '-' \
                                                        if x == '' else x)

alfagift_member['created_at'] = alfagift_member['created_at'].fillna('2019-01-01T23:08:53.393Z')
alfagift_member['created_at'] = alfagift_member['created_at'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)

# alfagift_member = alfagift_member[['account_card', 'sapa', 'created_at']]
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: 'sapa' if x != '-' else 'not_sapa')

alfagift_member['created_at_d'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m-%d")
alfagift_member['created_at_m'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m")

alfagift_member['account_card'] = alfagift_member['account_card'].astype('str')

In [ ]:
# alfagift_member_sel = alfagift_member[(alfagift_member['created_at_m'] == '2022-01') |
#                                       (alfagift_member['created_at_m'] == '2022-02') |
#                                       (alfagift_member['created_at_m'] == '2022-03') |
#                                       (alfagift_member['created_at_m'] == '2022-04') ]
# alfagift_member_sel['created_at_d'] = pd.to_datetime(alfagift_member_sel['created_at']).dt.strftime("%Y-%m-%d")

# alfagift_member_sel = alfagift_member.copy()

In [89]:
### creating label for training set


li_sus = []
for dr in pd.date_range('2021-12-01', '2022-04-20', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]
    #print(alfagift_member_sel_d.shape)


    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','count'))
    alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
        alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

    sus_by_ip = pd.merge(alfagift_member_sel_d_ip, alfagift_member_sel_d, on='createdFromIp', how='left')

    ## suspect by phone
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(createdFromIp_count=('phone_sub','count'))
    alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
        alfagift_member_sel_d_phone['createdFromIp_count'] >= 5].reset_index()

    sus_by_phone = pd.merge(alfagift_member_sel_d_phone, alfagift_member_sel_d, on='phone_sub', how='left')

    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    
    
    ## saving by concating suspect aspect
    sus_all = pd.concat([
        sus_by_ip[['account_card','email','phone']], 
        sus_by_phone[['account_card','email','phone']],
        alfagift_member_sel_d[alfagift_member_sel_d['member_status'] == 27][['account_card','email']],
        alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] > 4][['account_card','email']]
    ], axis=0)\
        .drop_duplicates()
    
    sus_all = sus_all[~sus_all['email'].str.contains('deliverysapa', case=False)]
    sus_all = sus_all[['account_card','phone','email']]
    
    
    ## inserting into table to check transaction
    df = sus_all[['account_card']]

    engine = create_engine(engine_stmt)


    dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
    df.to_sql('temp_sus_member', engine, index=False, 
                                          if_exists="replace", dtype=dtyp)
    
    
    q = '''
    
        SELECT 
            rto.TRO_DATE, rto.TRO_PONTA_ID
        FROM 
            temp_sus_member tsm
            INNER JOIN RPT_TRANS_OSHOP rto
            ON rto.TRO_PONTA_ID = tsm.ACCOUNT_CARD 
    
    '''
    con_alfabi = engine.connect()
    df_exclude = pd.read_sql_query(q, con_alfabi)
    df_exclude_g =  df_exclude.groupby('tro_ponta_id').agg(tro_date_count=('tro_date','nunique'))
    df_exclude_g = df_exclude_g[df_exclude_g['tro_date_count'] >2].reset_index()

    
    con_alfabi.close()
    
    sus_all = sus_all[~sus_all['account_card'].isin(df_exclude_g['tro_ponta_id'])]
    ### end inserting into table to check transaction
    li_sus.append(sus_all)


    #break
    sus_all.to_csv(
        '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_{}'.format(sel_created)
    )

    

df_sus_all = pd.concat(li_sus).drop_duplicates()
df_sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_all_exclude_trx.csv', index=False)

In [90]:
# alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] > 3.8]

In [245]:
string1 = "santi78951321santi"
string2 = ["santi12344", "aaaasanti"]


def seq_match(string1, li_string):
    # match = SequenceMatcher(None, string1, string2).find_longest_match()
    
#     sum_score = 0
    match_start = 0
    match_end = 0
    for string2 in li_string:
        match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

#         print(match)  # -> Match(a=0, b=15, size=9)
        
        if match.size >= 3:


            if ( string1.startswith(string1[match.a:match.a + match.size]) 
                and  string2.startswith(string2[match.b:match.b + match.size]) ) :
#                 print('match in front')
                
                match_start += 1
                
            if ( string1.endswith(string1[match.a:match.a + match.size]) 
                and string2.endswith(string2[match.b:match.b + match.size]) ):
#                 print('match in end')
                
                match_end += 1
        
#         print(match_start, match_end)
#         if match_end or match_start:
#             sum_score += 1
        
        
    return match_start, match_end
    
# seq_match(string1, string2)


# ### EXAMPLE USAGE OF sequence matcher

# x = 'santi'
# alfagift_member_test = alfagift_member_sel_d[(alfagift_member_sel_d['full_name'].str.startswith(x)) | 
#                          (alfagift_member_sel_d['full_name'].str.endswith(x))][['full_name']]

# alfagift_member_test['full_name_sCount'], alfagift_member_test['full_name_eCount'] = zip(*alfagift_member_test['full_name']\
#         .apply(lambda x: seq_match(x, alfagift_member_test['full_name'])))

# alfagift_member_test.head()


def add_features(alfagift_member_sel_d):
    
    alfagift_member_sel_d['full_name_sCount'], alfagift_member_sel_d['full_name_eCount'] = \
            zip(*alfagift_member_sel_d['full_name']\
            .apply(lambda x: seq_match(x, alfagift_member_sel_d['full_name'])))
    return alfagift_member_sel_d

def parallelize_dataframe(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [246]:
# ## creating feature for training set


# li_sus = []
# for dr in pd.date_range('2021-12-01', '2022-04-20', freq='D'):
#     sel_created = dr.strftime("%Y-%m-%d")

#     alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]
#     # print("{} {}".format(sel_created, alfagift_member_sel_d.shape))
#     alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
#     alfagift_member_sel_d['full_name'] = alfagift_member_sel_d['full_name'].str.lower()
    
#     ## suspect by IP
#     alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
#                             .agg(createdFromIp_count=('createdFromIp','count'))

#     alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
#                                      alfagift_member_sel_d_ip,
#                                      on='createdFromIp', how='left').fillna(0)

#     ## suspect by phone
#     alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
#                                 .agg(phoneSub_count=('phone_sub','count'))

#     alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
#                                      alfagift_member_sel_d_phone,
#                                      on='phone_sub', how='left').fillna(0)
    
    
#     alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
#     alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    
# #     alfagift_member_sel_d['nameMatch_count'] = alfagift_member_sel_d['full_name']\
# #     .apply(lambda x: len(process.extract(x, alfagift_member_sel_d['full_name'], 
# #             score_cutoff=70, limit=alfagift_member_sel_d.shape[0], scorer=fuzz.token_sort_ratio))-1)
 
#     alfagift_member_sel_d = parallelize_dataframe(alfagift_member_sel_d, add_features)


#     li_sus.append(alfagift_member_sel_d)
#     alfagift_member_sel_d.to_csv(
#     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily/{}'.format(sel_created))

#     break

# # df_sus_all = pd.concat(li_sus)
# # df_sus_all.to_csv(
# #     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full_withname1.csv'
# #     , index=False)

In [296]:
alfagift_member_sel_d.head()

,_id,member_status,full_name,gender,email,birth_date,phone,marital_status,account_card,createdFromIp,created_at,sapa,created_at_d,created_at_m,phone_sub,createdFromIp_count,phoneSub_count,score_gibberish,score_sus_email,full_name_sCount,full_name_eCount,full_name_sCount,full_name_eCount
0,{'$oid': '61a6bb091a64344ac66a1e0f'},26,nia,F,papikittyreborn@gmail.com,{'$date': '2006-01-10T00:00:00Z'},087851970483,S,9990013478276852,140.213.58.147,2021-12-01T00:00:09.097Z,not_sapa,2021-12-01,2021-12,087851970,1,1,3.23,0,1,2,1,1
1,{'$oid': '61a6bb40c655f22d7f5b759e'},26,erni ramayanti,F,syajayanira@yahoo.co.id,{'$date': '1983-12-01T00:00:00Z'},08127871033,M,9990010717145475,114.125.100.226,2021-12-01T00:01:04.98Z,not_sapa,2021-12-01,2021-12,081278710,1,1,2.33,0,1,9,1,1
2,{'$oid': '61a6bb54c655f22d7f5b75a2'},26,hendra husain,F,hendrahusain640@gmail.com,{'$date': '1997-04-29T17:00:00Z'},082395244457,M,9990013478276966,103.10.64.13,2021-12-01T00:01:24.259Z,not_sapa,2021-12-01,2021-12,082395244,1,1,2.55,0,2,1,1,1
3,{'$oid': '61a6bb621a64344ac66a1e18'},26,ismianti dr,F,iyanjuni02@gmail.com,{'$date': '1994-01-10T00:00:00Z'},089614735504,M,9990013478277053,180.252.94.165,2021-12-01T00:01:38.471Z,not_sapa,2021-12-01,2021-12,089614735,1,1,2.27,0,1,1,1,1
4,{'$oid': '61a6bb6ec655f22d7f5b75a5'},26,oktaviana eka wulansari,F,oktvneka@gmail.com,{'$date': '2003-10-26T17:00:00Z'},088245644936,S,9990013478277167,114.122.202.48,2021-12-01T00:01:50.659Z,not_sapa,2021-12-01,2021-12,088245644,1,1,4.13,0,1,6,1,1


In [300]:
alfagift_member_sel_d.shape

(7883, 23)

In [305]:
alfagift_member_sel_d.reset_index(drop=True).head()

,_id,member_status,full_name,gender,email,birth_date,phone,marital_status,account_card,createdFromIp,created_at,sapa,created_at_d,created_at_m,phone_sub,createdFromIp_count,phoneSub_count,score_gibberish,score_sus_email,full_name_sCount,full_name_eCount,full_name_sCount,full_name_eCount
0,{'$oid': '61a6bb091a64344ac66a1e0f'},26,nia,F,papikittyreborn@gmail.com,{'$date': '2006-01-10T00:00:00Z'},087851970483,S,9990013478276852,140.213.58.147,2021-12-01T00:00:09.097Z,not_sapa,2021-12-01,2021-12,087851970,1,1,3.23,0,1,2,1,1
1,{'$oid': '61a6bb40c655f22d7f5b759e'},26,erni ramayanti,F,syajayanira@yahoo.co.id,{'$date': '1983-12-01T00:00:00Z'},08127871033,M,9990010717145475,114.125.100.226,2021-12-01T00:01:04.98Z,not_sapa,2021-12-01,2021-12,081278710,1,1,2.33,0,1,9,1,1
2,{'$oid': '61a6bb54c655f22d7f5b75a2'},26,hendra husain,F,hendrahusain640@gmail.com,{'$date': '1997-04-29T17:00:00Z'},082395244457,M,9990013478276966,103.10.64.13,2021-12-01T00:01:24.259Z,not_sapa,2021-12-01,2021-12,082395244,1,1,2.55,0,2,1,1,1
3,{'$oid': '61a6bb621a64344ac66a1e18'},26,ismianti dr,F,iyanjuni02@gmail.com,{'$date': '1994-01-10T00:00:00Z'},089614735504,M,9990013478277053,180.252.94.165,2021-12-01T00:01:38.471Z,not_sapa,2021-12-01,2021-12,089614735,1,1,2.27,0,1,1,1,1
4,{'$oid': '61a6bb6ec655f22d7f5b75a5'},26,oktaviana eka wulansari,F,oktvneka@gmail.com,{'$date': '2003-10-26T17:00:00Z'},088245644936,S,9990013478277167,114.122.202.48,2021-12-01T00:01:50.659Z,not_sapa,2021-12-01,2021-12,088245644,1,1,4.13,0,1,6,1,1


In [301]:
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','cumcount'))

In [303]:
alfagift_member_sel_d_ip.reset_index(drop=True).head()

,createdFromIp_count
0,0
1,0
2,0
3,0
4,0


In [ ]:
import time
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count

def square(dr):


    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]\
                            .iloc[:,2:].reset_index(drop=True)
    
    
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d['full_name'] = alfagift_member_sel_d['full_name'].str.lower()

    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','cumcount')).reset_index(drop=True)

    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, 
                                     alfagift_member_sel_d_ip]
                                     ,axis=1)

    ## suspect by phone
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(phoneSub_count=('phone_sub','cumcount')).reset_index(drop=True)

    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, 
                                     alfagift_member_sel_d_phone]
                                     ,axis=1)
    
    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    

    li_name_count = []
    for idx, row in alfagift_member_sel_d.iterrows():
        cur_name = row['full_name']

        compare_name = list(alfagift_member_sel_d.iloc[0:idx+1,:]['full_name'])


        res_compare = seq_match(cur_name, compare_name)
        li_name_count.append(res_compare)


    df_count_name = pd.DataFrame(li_name_count, columns=['full_name_sCount', 'full_name_eCount'])
    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, df_count_name], axis=1)

    alfagift_member_sel_d.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily/{}'.format(sel_created))


    return 0


start = timer()

print(f'starting computations on {cpu_count()} cores')

values = pd.date_range('2021-12-30', '2022-04-20', freq='D')

with Pool() as pool:
    res = pool.map(square, values)
    #print(res)

end = timer()
print(f'elapsed time: {end - start}')

starting computations on 64 cores


In [294]:
## creating feature using cumulative





In [293]:
# alfagift_member_sel_d_ip1 = alfagift_member_sel_d.groupby('createdFromIp').agg(createdFromIp_count=('createdFromIp','cumcount'))

In [290]:
# from datetime import datetime
# start_time = datetime.now()

# res = seq_match('akhiyar',list(alfagift_member_sel_d.iloc[:,:]['full_name']))
# end_time = datetime.now()
# print('Duration: {}'.format(end_time - start_time))

In [ ]:
# # possible 
# ## need to check score
# afghj56diskazani
# afghj56rdiyanto
# sfghj56upriyatno
# dfghj56wiyanti
# dfghj56wiyanti
# wfgh568ahyusaputra

In [185]:
virgo_abuseinject = pd.read_excel(
    '/home/server/gli-data-science/ferina/Virgo_Abuse.xlsx'
    ,dtype=object,sheet_name='Inject', names=['account_card'])
# virgo_abuseuse = pd.read_excel(
#     '/home/server/gli-data-science/ferina/Virgo_Abuse.xlsx'
#     ,dtype=object,sheet_name='Use', names=['account_card'])

## load feature and label 
df_sus_all = pd.read_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full_withname1.csv'
    , dtype='object')
# df_sus_all = df_sus_all[(df_sus_all['created_at_d'] >= '2021-12-01') 
#                         & (df_sus_all['created_at_d'] <= '2022-04-17')]


sus_label = pd.read_csv('./suspect_fraud/sus_all_exclude_trx.csv', dtype='object', usecols=[0])


df_label = pd.concat([sus_label, virgo_abuseinject, virgo_abuseuse]).drop_duplicates()
df_label['label'] = 1

## merging feature with suspect label
df_train = pd.merge(df_sus_all, df_label, on='account_card', how='left')
df_train['label'] = df_train['label'].fillna(0)

df_train['score_gibberish'] = df_train['score_gibberish'].astype(float).astype(int)
df_train['score_sus_email'] = df_train['score_sus_email'].astype(int)
df_train['createdFromIp_count'] = df_train['createdFromIp_count'].astype(int)
df_train['phoneSub_count'] = df_train['phoneSub_count'].astype(int)
df_train['full_name_sCount'] = df_train['full_name_sCount'].astype(int)
df_train['full_name_eCount'] = df_train['full_name_eCount'].astype(int)

In [186]:
wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',
'full_name_sCount',
'full_name_eCount'
]
X = df_train[wrap_col]
y = df_train[['label']]

In [206]:
#Initiating the Agglomerative Clustering model 
AC = KMeans(n_clusters=9, random_state=0)
# AC = KModes(n_clusters=5, init='Huang', n_init=5, verbose=1)

# fit model and predict clusters without PCA
yhat_AC = AC.fit_predict(X)

df_train["Clusters"]= yhat_AC


In [207]:
df_train["Clusters"].value_counts()

0    2168317
3     310526
6      76459
8      34063
4      14996
2       7571
1       7337
5       6699
7       1750
Name: Clusters, dtype: int64

In [238]:
df_train[df_train['Clusters'] == 8].iloc[:,15:-2]\
.describe()

,createdFromIp_count,phoneSub_count,score_gibberish,score_sus_email,full_name_sCount,full_name_eCount
count,34063.00,34063.00,34063.00,34063.00,34063.00,34063.00
mean,3.78,10.31,4.15,0.05,53.23,6.95
std,6.86,13.91,1.33,0.21,20.06,13.72
min,1.00,1.00,0.00,0.00,14.00,1.00
25%,1.00,1.00,3.00,0.00,36.00,1.00
50%,1.00,5.00,4.00,0.00,49.00,1.00
75%,3.00,13.00,5.00,0.00,65.00,4.00
max,81.00,106.00,9.00,1.00,113.00,91.00


In [237]:
# df_train[(df_train['Clusters'] == 8)]['created_at_d'].value_counts()

In [236]:
df_train[(df_train['Clusters'] == 6) & (df_train['created_at_d'] > '2022-04-05')]

,_id,member_status,full_name,gender,email,birth_date,phone,marital_status,account_card,createdFromIp,created_at,sapa,created_at_d,created_at_m,phone_sub,createdFromIp_count,phoneSub_count,score_gibberish,score_sus_email,full_name_sCount,full_name_eCount,label,Clusters
2486467,{'$oid': '624cfa5c595d5660b6db6c53'},26,anggi kp,F,kleptomaniakygdikasihi@gmail.com,{'$date': '1990-02-12T00:00:00Z'},083832939830,M,9990013751925080,110.138.86.59,2022-04-06T02:26:36.015Z,not_sapa,2022-04-06,2022-04,083832939,38,1,2,0,1,1,1.00,6
2486550,{'$oid': '624cfc4a5bb9f4497a6e9a22'},26,gunah,M,werdi23@yahoo.com,{'$date': '1993-03-18T00:00:00Z'},085697053452,M,9990013751934193,112.215.243.12,2022-04-06T02:34:50.69Z,not_sapa,2022-04-06,2022-04,085697053,33,1,2,0,1,5,1.00,6
2486611,{'$oid': '624cfdadae855d51cd3d9842'},26,ajeng vb,F,doipilahpilihaje@gmail.com,{'$date': '1988-08-05T00:00:00Z'},083865471967,S,9990013751940851,110.138.86.59,2022-04-06T02:40:45.174Z,not_sapa,2022-04-06,2022-04,083865471,38,5,2,0,1,1,1.00,6
2486717,{'$oid': '624d002bafcd8e34e346b407'},26,indah,F,jiuhuu@yahoo.com,{'$date': '1994-06-12T00:00:00Z'},083897807336,M,9990013751953332,112.215.243.12,2022-04-06T02:51:23.483Z,not_sapa,2022-04-06,2022-04,083897807,33,1,3,0,2,3,1.00,6
2486731,{'$oid': '624d0060a1638559d636acae'},26,helmy op,M,dimanamanaadadia@gmail.com,{'$date': '1989-10-11T00:00:00Z'},083857274529,M,9990013751954929,110.138.86.59,2022-04-06T02:52:16.984Z,not_sapa,2022-04-06,2022-04,083857274,38,3,2,0,1,1,1.00,6
2486840,{'$oid': '624d0256965c4c2c95e5aeea'},26,renata,F,rewd23@yahoo.com,{'$date': '1995-05-08T00:00:00Z'},083899227819,M,9990013751966511,112.215.243.12,2022-04-06T03:00:38.707Z,not_sapa,2022-04-06,2022-04,083899227,33,1,4,0,4,2,1.00,6
2486886,{'$oid': '624d0345ae855d51cd3d98f7'},26,nabila sh,F,sepedalagulodah@gmail.com,{'$date': '1983-05-17T00:00:00Z'},083838770977,M,9990013751972000,110.138.86.59,2022-04-06T03:04:37.915Z,not_sapa,2022-04-06,2022-04,083838770,38,3,2,0,2,1,1.00,6
2487060,{'$oid': '624d06f85bb9f4497a6e9b37'},26,dul ks,M,disanasenangdisinisenang@gmail.com,{'$date': '1992-10-12T00:00:00Z'},083837078621,S,9990013751994051,110.138.86.59,2022-04-06T03:20:24.368Z,not_sapa,2022-04-06,2022-04,083837078,38,1,1,0,1,1,1.00,6
2487172,{'$oid': '624d095c965c4c2c95e5affc'},26,fitta,F,fiiitui23@yahoo.com,{'$date': '1996-06-15T00:00:00Z'},083893432616,M,9990013752008690,112.215.243.12,2022-04-06T03:30:36.383Z,not_sapa,2022-04-06,2022-04,083893432,33,1,3,0,4,1,1.00,6
2487175,{'$oid': '624d09709c150f3670e1ec80'},26,novi bg,F,santuyajadulubisku@gmail.com,{'$date': '1986-10-12T00:00:00Z'},083839585131,M,9990013752008932,110.138.86.59,2022-04-06T03:30:56.217Z,not_sapa,2022-04-06,2022-04,083839585,38,1,2,0,3,1,1.00,6


In [96]:
# regressor = DecisionTreeClassifier(random_state=0, max_depth=4)
# regressor.fit(X,y)


# dump(regressor, './model/regis_dtree')
# r = export_text(regressor, feature_names=list(X), decimals=0, spacing=6)
# print("prediction rule \n\n{}".format(r))

In [123]:
clf = LogisticRegression(random_state=0)
clf.fit(X.values, y.values.ravel())
dump(clf, './model/regis_logreg_withname1.pkl')

['./model/regis_logreg_withname1.pkl']

In [124]:
X.values[3]

array(['2.2673275613367374', '0', '1', '1', '1'], dtype=object)

In [148]:
li_input = [[
    2,
    0,
    5,
    0,
    100,
]]
X_test = np.array(li_input)
clf.predict_proba(X_test)

array([[0.97925527, 0.02074473]])

In [126]:
clf.coef_

array([[1.7679208 , 0.4569415 , 0.26816545, 0.59626202, 0.00333049]])

In [74]:
# ## for concating feature and bobot
# pd.concat([pd.DataFrame(list(X), columns=['variabel']), 
#            pd.DataFrame(pd.Series(clf.coef_[0]), columns=['bobot'])], 1)\
# .append({'variabel':'intercept','bobot':clf.intercept_[0]},ignore_index=True)

wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',
'full_name_sCount',
# 'full_name_eCount'
]
res_pred = pd.concat([
    pd.DataFrame(wrap_col, columns=['variabel']), 
    pd.DataFrame(clf.coef_[0], columns=['bobot']),
    pd.DataFrame(X_test[0], columns=['feature'])      
], 1)
# res_pred['feature'] = res_pred['feature'].astype(int) 
# res_pred['b*f'] = res_pred['bobot'] * res_pred['feature']



# dict_score_parameter = {}
# for idx, row in res_pred.iterrows():
#     dict_score_parameter[row['variabel']] = row['b*f']
res_pred

,variabel,bobot,feature
0,score_gibberish,-0.49,3.60
1,score_sus_email,-0.77,1.00
2,createdFromIp_count,0.01,6.00
3,phoneSub_count,0.03,6.00
4,full_name_sCount,0.01,100.00


In [ ]:
# model_clf = load('./model/regis_logreg.pkl')


# X_test = pd.DataFrame()
# X_test['score_gibberish'] = [1]
# X_test['score_sus_email'] = [0]
# X_test['createdFromIp_count'] = [0]
# X_test['phoneSub_count'] = [0]

# score = model_clf.predict_proba(X_test)
# score[0][-1]


# res_pred = pd.concat([
#     pd.DataFrame(clf.feature_names_in_, columns=['variabel']), 
#     pd.DataFrame(clf.coef_[0], columns=['bobot']),
#     pd.DataFrame(X_test.values[0], columns=['feature'])      
# ], 1)
# res_pred['b*f'] = res_pred['bobot'] * res_pred['feature']

# dict_score_parameter = {}
# for idx, row in res_pred.iterrows():
#     dict_score_parameter[row['variabel']] = row['b*f']
# dict_score_parameter